# Smart signatures
#### 06.1 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-12

* Write and deploy smart Signatures

## Setup
See notebook 04.1, the lines below will always automatically load ...
* The functions in `algo_util.py`
* The accounts MyAlgo, Alice and Bob
* The Purestake credentials

In [2]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [3]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

In [4]:
from pyteal import *

In [5]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [6]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

HITPAAJ4HKANMP6EUYASXDUTCL653T7QMNHJL5NODL6XEGBM4KBLDJ2D2E
O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
5GIOBOLZSQEHTNNXWRJ6RGNPGCKWYJYUZZKY6YXHJVKFZXRB2YLDFDVH64


#### Check Purestake API

In [7]:
last_block = algod_client.status()["last-round"]
print(f"Last committed block is: {last_block}")

Last committed block is: 19795213


## The Donation Escrow
A classical smart contact. **Alice** donates to **Bob** (and only to **Bob**).

**Bob** can decide when to ask for the money.

##### Step 1: The programmer writes down the conditions as a PyTeal program

In [9]:
escrow_pyteal =  (
    Txn.receiver() == Addr(Bob["public"])            # Receipient must be Bob
)                                                    # Encode addresses using Addr()

# Security missing ... do not copy-paste

##### Step 2: Compile PyTeal -> Teal

In [10]:
escrow_teal = compileTeal(escrow_pyteal, Mode.Signature, version=3)
print(escrow_teal)

#pragma version 3
txn Fee
int 1000
<=
txn TypeEnum
int pay
==
global GroupSize
int 1
==
&&
txn RekeyTo
global ZeroAddress
==
&&
&&
txn Receiver
addr O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
==
&&


##### Step 3: Compile Teal -> Bytecode for AVM

In [11]:
# compile Teal -> Bytecode
Escrow = algod_client.compile(escrow_teal)
Escrow

{'hash': 'AXOSZ2SI2UW7R5ELGRGWPIKG3MTOC3QJ5KVUTRLBYEDZ5O5X4WDVSNKRDQ',
 'result': 'AyAC6AcBJgEgdqS4vVxHJWo4WDGUc9BFSkvXROVf/YdzZcTD89HfAcYxASIOMRAjEjIEIxIQMSAyAxIQEDEHKBIQ'}

##### Step 4: Alice funds and deploys the smart signature

In [13]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(1.5*1e6)
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, receiver=Escrow['hash'], amt=amt)

# Step 2+3: sign and sen
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19795244.
Waiting for round 19795244 to finish.
Waiting for round 19795245 to finish.
Transaction 6LVEHDH6KEKQC2OURP3PRGFVIXD33HJYNCQSNH4I3VJ35NOTIHIQ confirmed in round 19795246.


In [15]:
# Smart Signature is now funded
print('https://testnet.algoexplorer.io/address/'+ Escrow['hash'])

https://testnet.algoexplorer.io/address/AXOSZ2SI2UW7R5ELGRGWPIKG3MTOC3QJ5KVUTRLBYEDZ5O5X4WDVSNKRDQ


##### Step 5: Alice informs Bob

In [16]:
print("Alice communicates to Bob the following")
print("Compiled smart signature:", Escrow['result'])
print("Address of smart signature: ", Escrow['hash'])

Alice communicates to Bob the following
Compiled smart signature: AyAC6AcBJgEgdqS4vVxHJWo4WDGUc9BFSkvXROVf/YdzZcTD89HfAcYxASIOMRAjEjIEIxIQMSAyAxIQEDEHKBIQ
Address of smart signature:  AXOSZ2SI2UW7R5ELGRGWPIKG3MTOC3QJ5KVUTRLBYEDZ5O5X4WDVSNKRDQ


#### Step 6: Bob proposes a transaction to the smart signature
* Again he proposes a payment from the cashmachine to **himself**
* The payment transaction is signed by the smart signature, **if the conditions are fullfilled** (correct recipient)

In [18]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(0.1*1e6)
txn = PaymentTxn(sender=Escrow['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Escrow['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19795265.
Waiting for round 19795265 to finish.
Transaction 6RK62F676OSFS3RSXE5VYH3GPWHUANVZQA7FLSIMBCWKCJBWN5LQ confirmed in round 19795266.


### Exercise
* Repeat Step 6. Can Bob withdraw several times?
* Write the transaction(s) for Charlie to try to withdraw from the Escrow

In [ ]:
# Python code goes here